In [6]:
using PrettyTables

function RK38(f, x0, y0, N, h)
    approx = [x0 y0]
    x = x0
    w = y0
    
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h/3, w + h * (1/3) * k1)
        k3 = f(x + 2*h/3, w + h * ((-1/3) * k1 + 1 * k2))
        k4 = f(x + h, w + h * (1 * k1 + (-1) * k2 + 1 * k3))
        
        w = w + h * ((1/8) * k1 + (3/8) * k2 + (3/8) * k3 + (1/8) * k4)
        x += h
        approx = vcat(approx, [x w])
    end
    return approx
end

f(x, y) = -y + x * sqrt(y)
x0 = 2
y0 = 2
h = 0.1
xf = 3
N = Int((xf - x0) / h)

approx = RK38(f, x0, y0, N, h)

pretty_table(approx, column_labels = ["x", "y(x)"])

┌─────┬─────────┐
│   x │    y(x) │
├─────┼─────────┤
│ 2.0 │     2.0 │
│ 2.1 │ 2.08872 │
│ 2.2 │ 2.18931 │
│ 2.3 │ 2.30197 │
│ 2.4 │ 2.42693 │
│ 2.5 │ 2.56445 │
│ 2.6 │ 2.71483 │
│ 2.7 │ 2.87838 │
│ 2.8 │ 3.05542 │
│ 2.9 │ 3.24628 │
│ 3.0 │ 3.45129 │
└─────┴─────────┘


In [7]:
using PrettyTables

function RK4(f, x0, y0, N, h)
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h / 2, w + h * k1 / 2)
        k3 = f(x + h / 2, w + h * k2 / 2)
        k4 = f(x + h, w + h * k3)
        w = w + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        x += h
        approx = vcat(approx, [x w])
    end
    return approx
end

function Newton(f, df, x0, eps, N)
    i = 1
    while i <= N
        x = x0 - f(x0) / df(x0)
        if abs(x - x0) < eps
            return x
        else
            x0 = x
        end
        i = i + 1
    end
end

function BDF2(f, fy, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    w0 = y0
    
    RK_approx = RK4(f, x0, y0, 1, h)
    w1 = RK_approx[2, 2]
    approx = [x0 w0; x1 w1]
    
    for i in 1:N-1
        g(w2) = w2 - (4/3)*w1 + (1/3)*w0 - (2*h/3)*f(x2, w2)
        dg(w2) = 1 - (2*h/3)*fy(x2, w2)
        
        w2 = Newton(g, dg, w1, h^3, 10000)
        approx = vcat(approx, [x2 w2])
        
        x0 = x1
        x1 = x2
        x2 += h
        w0 = w1
        w1 = w2
    end
    return approx
end

f(x, y) = (2 - 2*x*y) / (x^2 + 1)
fy(x, y) = -2*x / (x^2 + 1)

a = 0
b = 1
y0 = 1
h = 0.01
N = Int((b - a) / h)

approx = BDF2(f, fy, a, y0, N, h)

println("Using bdf22 with  $h:")
println("y(1) ~ $(approx[N + 1, 2])")

println("\first points:")
pretty_table(approx[1:6, :], column_labels = ["x", "y(x)"])

println("\nlast points:")
pretty_table(approx[end-5:end, :], column_labels = ["x", "y(x)"])

Using bdf22 with  0.01:
y(1) ~ 1.5000512129596761
irst points:
┌──────┬─────────┐
│    x │    y(x) │
├──────┼─────────┤
│  0.0 │     1.0 │
│ 0.01 │  1.0199 │
│ 0.02 │ 1.03958 │
│ 0.03 │ 1.05904 │
│ 0.04 │ 1.07827 │
│ 0.05 │ 1.09724 │
└──────┴─────────┘

last points:
┌──────┬─────────┐
│    x │    y(x) │
├──────┼─────────┤
│ 0.95 │ 1.52436 │
│ 0.96 │ 1.51962 │
│ 0.97 │ 1.51481 │
│ 0.98 │ 1.50995 │
│ 0.99 │ 1.50503 │
│  1.0 │ 1.50005 │
└──────┴─────────┘


In [8]:
using PrettyTables

function Euler(f, x0, y0, N, h)
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        w = w + h * f(x, w)
        x += h
        approx = vcat(approx, [x w])
    end
    return approx
end

function RK4(f, x0, y0, N, h)
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h / 2, w + h * k1 / 2)
        k3 = f(x + h / 2, w + h * k2 / 2)
        k4 = f(x + h, w + h * k3)
        w = w + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        x += h
        approx = vcat(approx, [x w])
    end
    return approx
end

function AB4_Euler(f, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    
    Euler_approx = Euler(f, x0, y0, 3, h)
    w1 = Euler_approx[2, 2]
    w2 = Euler_approx[3, 2]
    w3 = Euler_approx[4, 2]
    
    approx = [x0 w0; x1 w1; x2 w2; x3 w3]
    
    for i in 1:N-3
        x4 = x3 + h
        w4 = w3 + h / 24 * (55 * f(x3, w3) - 59 * f(x2, w2) + 37 * f(x1, w1) - 9 * f(x0, w0))
        approx = vcat(approx, [x4 w4])
        x0 = x1
        x1 = x2
        x2 = x3
        x3 = x4
        w0 = w1
        w1 = w2
        w2 = w3
        w3 = w4
    end
    return approx
end

function AB4_RK4(f, x0, y0, N, h)
    x1 = x0 + h
    x2 = x1 + h
    x3 = x2 + h
    w0 = y0
    RK_approx = RK4(f, x0, y0, 3, h)
    w1 = RK_approx[2, 2]
    w2 = RK_approx[3, 2]
    w3 = RK_approx[4, 2]
    approx = [x0 w0; x1 w1; x2 w2; x3 w3]
    for i in 1:N-3
        x4 = x3 + h
        w4 = w3 + h / 24 * (55 * f(x3, w3) - 59 * f(x2, w2) + 37 * f(x1, w1) - 9 * f(x0, w0))
        approx = vcat(approx, [x4 w4])
        x0 = x1
        x1 = x2
        x2 = x3
        x3 = x4
        w0 = w1
        w1 = w2
        w2 = w3
        w3 = w4
    end
    return approx
end

f(x, y) = y
exact(x) = exp(x)
a = 0
b = 1
y0 = 1

println("AB4 with euler init:")
errors_euler = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    approx = AB4_Euler(f, a, y0, N, h)
    y = approx[N + 1, 2]
    e = abs(y - exact(b))
    if i == 1
        errors_euler = [h e]
    else
        errors_euler = vcat(errors_euler, [h e])
    end
end
pretty_table(errors_euler, column_labels = ["h", "Error"])

println("\nab4 vs. rk4:")
errors_rk4 = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    approx = AB4_RK4(f, a, y0, N, h)
    y = approx[N + 1, 2]
    e = abs(y - exact(b))
    if i == 1
        errors_rk4 = [h e]
    else
        errors_rk4 = vcat(errors_rk4, [h e])
    end
end
pretty_table(errors_rk4, column_labels = ["h", "Error"])

AB4 with euler init:
┌────────┬─────────────┐
│      h │       Error │
├────────┼─────────────┤
│    0.1 │   0.0381573 │
│   0.01 │ 0.000405136 │
│  0.001 │  4.07482e-6 │
│ 0.0001 │  4.07716e-8 │
└────────┴─────────────┘

ab4 vs. rk4:
┌────────┬─────────────┐
│      h │       Error │
├────────┼─────────────┤
│    0.1 │  5.73893e-5 │
│   0.01 │  9.05699e-9 │
│  0.001 │ 9.41913e-13 │
│ 0.0001 │ 1.19904e-14 │
└────────┴─────────────┘


In [9]:
using PrettyTables

function RK4(f, x0, y0, N, h)
    approx = [x0 y0]
    x = x0
    w = y0
    for i in 1:N
        k1 = f(x, w)
        k2 = f(x + h / 2, w + h * k1 / 2)
        k3 = f(x + h / 2, w + h * k2 / 2)
        k4 = f(x + h, w + h * k3)
        w = w + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        x += h
        approx = vcat(approx, [x w])
    end
    return approx
end

function AB2AM1(f, x0, y0, N, h)
    
    x1 = x0 + h
    w0 = y0
    
    RK_approx = RK4(f, x0, y0, 1, h)
    w1 = RK_approx[2, 2]
    approx = [x0 w0; x1 w1]
    
    for i in 1:N-1
        x2 = x1 + h
        
        f0 = f(x0, w0)
        f1 = f(x1, w1)
        
        w2p = w1 + h * (1.5 * f1 - 0.5 * f0)
        
        w2 = w1 + h / 2 * (f1 + f(x2, w2p))
        
        approx = vcat(approx, [x2 w2])
        
        x0 = x1
        x1 = x2
        w0 = w1
        w1 = w2
    end
    return approx
end

f(x, y) = x + y
exact_value = 2 * exp(1) - 2

a = 0
b = 1
y0 = 1

println("Exact value: y(1) = 2e - 2 = $exact_value\n")

errors = []
for i in 1:4
    N = 10^i
    h = (b - a) / N
    approx = AB2AM1(f, a, y0, N, h)
    y = approx[N + 1, 2]
    e = abs(y - exact_value)
    if i == 1
        errors = [h y e]
    else
        errors = vcat(errors, [h y e])
    end
end

pretty_table(errors, column_labels = ["h", "y(1) estimate", "Error"])

Exact value: y(1) = 2e - 2 = 3.43656365691809

┌────────┬───────────────┬────────────┐
│      h │ y(1) estimate │      Error │
├────────┼───────────────┼────────────┤
│    0.1 │       3.43954 │ 0.00297148 │
│   0.01 │       3.43661 │ 4.35203e-5 │
│  0.001 │       3.43656 │ 4.51238e-7 │
│ 0.0001 │       3.43656 │ 4.52859e-9 │
└────────┴───────────────┴────────────┘
